# ⚖️ 法律领域大模型 SFT 数据构造流水线
本 Notebook 整合了从 **PDF 原文提取** 到 **多样化指令生成 (Instruct Tuning)** 的完整流程。

### 1. 环境准备
安装必要的依赖包。

In [ ]:
pip install pdfplumber tqdm openai

### 2. 第一阶段：PDF 数据清洗与解析
该部分负责读取 PDF 文件，移除页眉页脚、页码，并利用正则提取法律条文。

In [ ]:
import pdfplumber
import os
import re
import json
from tqdm.notebook import tqdm

# --- 配置路径 ---
RAW_DATA_DIR = '../data/raw' 
PROCESSED_DATA_DIR = '../data/processed'
os.makedirs(PROCESSED_DATA_DIR, exist_ok=True)

def clean_text_smart(text):
    if not text: return ""
    # A. 去除参考文献引用标号
    text = re.sub(r'\[\s*\d+(?:[-–,]\d+)*\s*\]', '', text)
    text = re.sub(r'［\s*\d+(?:[-–,]\d+)*\s*］', '', text)
    # B. 去除嵌在文本中间的页码
    text = re.sub(r'(?:^|\s|\\n)[-—–－]\s*\d+\s*[-—–－](?=\s|\\n|$)', ' ', text)
    # C. 去除孤立的行级页码
    lines = text.split('\n')
    cleaned_lines = [line for line in lines if not re.fullmatch(r'[-—–－\s\d]+', line.strip())]
    text = '\n'.join(cleaned_lines)
    # D. 修复中文断词
    pattern_broken_zh = r'([\u4e00-\u9fa5])\s+([\u4e00-\u9fa5])'
    text = re.sub(pattern_broken_zh, r'\1\2', text)
    text = re.sub(pattern_broken_zh, r'\1\2', text) 
    # E. 规范化空白字符
    text = re.sub(r'[ \t\r\f]+', ' ', text) 
    return text.strip()

def process_legal_doc(file_path):
    filename = os.path.basename(file_path)
    full_text = ""
    try:
        with pdfplumber.open(file_path) as pdf:
            for page in tqdm(pdf.pages, desc=f"解析 {filename}", leave=False):
                width, height = page.width, page.height
                bbox = (0, height * 0.05, width, height * 0.95)
                try:
                    page_crop = page.crop(bbox=bbox)
                    text = page_crop.extract_text()
                    if text: full_text += "\n" + text
                except: continue
    except Exception as e:
        print(f"读取失败: {e}")
        return []
    
    full_text = clean_text_smart(full_text)
    pattern = r"(第[0-9零一二三四五六七八九十百千]+条[\s\S]*?)(?=第[0-9零一二三四五六七八九十百千]+条|$)"
    matches = re.findall(pattern, full_text)
    
    return [{"source": filename, "type": "legal_article", "content": re.sub(r'\s+', ' ', m).strip()} 
            for m in matches if len(m) > 15]

In [ ]:
# 执行解析循环
if os.path.exists(RAW_DATA_DIR):
    files = [f for f in os.listdir(RAW_DATA_DIR) if f.lower().endswith('.pdf')]
    all_chunks = []
    for filename in tqdm(files, desc="总解析进度"):
        if "法" in filename:
            chunks = process_legal_doc(os.path.join(RAW_DATA_DIR, filename))
            all_chunks.extend(chunks)
    
    # 保存中间结果
    output_path = os.path.join(PROCESSED_DATA_DIR, 'raw_chunks.jsonl')
    with open(output_path, 'w', encoding='utf-8') as f:
        for chunk in all_chunks:
            f.write(json.dumps(chunk, ensure_ascii=False) + '\n')
    print(f"✅ 清洗完成，共得到 {len(all_chunks)} 条法条数据。")
else:
    print("❌ 未找到 raw 目录，请检查路径。")

### 3. 第二阶段：多样化指令数据生成 (SFT)
使用 LLM 将法条转化为：**案例分析、文书起草、概念解释**。

In [ ]:
import random
import time
from openai import OpenAI

# --- 填写你的 API 配置 ---
API_KEY = "你的API密钥"
BASE_URL = "https://api.siliconflow.cn/v1"
MODEL_NAME = "deepseek-ai/DeepSeek-V3"

client = OpenAI(api_key=API_KEY, base_url=BASE_URL)

# 提示词模板 (Case/Doc/Concept)
PROMPTS = {
    "case_analysis": "你是一位资深律师。请阅读法条：{content}。请构造一个包含多方冲突的咨询案例。User: 描述案情。Assistant: <思考过程>分析逻辑 + <法律建议>结论。",
    "doc_drafting": "你是一位律师。请根据法条：{content}。构造 User: 要求起草相关文书。Assistant: <思考过程>要点 + <文书正文>内容。",
    "concept_explain": "你是一位教授。请根据法条：{content}。构造 User: 小白提问概念。Assistant: <思考过程>拆解 + <通俗解释>举例。"
}

def generate_sft_data(chunk):
    task_type = random.choices(["case_analysis", "doc_drafting", "concept_explain"], weights=[0.6, 0.2, 0.2])[0]
    prompt_tpl = PROMPTS[task_type]
    
    try:
        response = client.chat.completions.create(
            model=MODEL_NAME,
            messages=[
                {"role": "system", "content": "你是一个法律专家数据构造助手。请返回 JSON 格式，包含 instruction 和 output 两个字段。"},
                {"role": "user", "content": prompt_tpl.format(content=chunk['content'])}
            ],
            response_format={"type": "json_object"}
        )
        data = json.loads(response.choices[0].message.content)
        return {
            "instruction": data.get("instruction", ""),
            "output": data.get("output", ""),
            "task_type": task_type,
            "source": chunk.get('source')
        }
    except: return None

In [ ]:
# 执行生成
sft_items = []
test_chunks = all_chunks[:10]  # 先测试前10条

for chunk in tqdm(test_chunks, desc="LLM 生成中"):
    item = generate_sft_data(chunk)
    if item: sft_items.append(item)
    time.sleep(0.2)

# 保存最终 SFT 数据
sft_path = os.path.join(PROCESSED_DATA_DIR, 'domain_expert_sft.jsonl')
with open(sft_path, 'w', encoding='utf-8') as f:
    for item in sft_items:
        f.write(json.dumps(item, ensure_ascii=False) + '\n')

print(f"✅ 生成完毕！文件已保存至: {sft_path}")